In [1]:
#!pip install git+https://github.com/huggingface/transformers sentencepiece datasets
#!pip install PyMuPDF

In [2]:
from tkinter import *
from tkinter import filedialog
import fitz
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

In [3]:
TITLE_FONT = "Heebo"
TITLE_COLOR = "#800080"
WORD_FONT = "Heebo"
WORD_COLOR = "#3A003A"
BACKGROUND_COLOR = "#B09FCA"
BTN_BACKGROUND_COLOR = "#3A003A"
LANGUAGE = "ro"

In [4]:
def text_speech():
    # Use tkinter filedialog to select a PDF file
    pdf_file = filedialog.askopenfilename(
        initialdir="/",
        title="Select PDF",
        filetypes=(("pdf files", "*.pdf"), ("all files", "*.*"))
    )
    
    #Opening the file
    doc = fitz.open(pdf_file)
    text = ""
    for page in doc:
        text+=page.get_text()
    print(text)
    
    print('Starting with the speech generation')
    processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
    model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
    vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
    
    inputs = processor(text=text, return_tensors="pt")

    # load xvector containing speaker's voice characteristics from a dataset
    embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    sf.write("speech_from_text.wav", speech.numpy(), samplerate=16000)
    print('The audio was generated. See You!!')

In [5]:
# Creating Windows and Labels with Tkinter
root = Tk()
root.title('Text to Speech')
root.config(padx=25, pady=25, bg=BACKGROUND_COLOR)
title_label = Label(text="Text to Speech", font=(TITLE_FONT, 40, "bold"), fg=TITLE_COLOR, bg=BACKGROUND_COLOR)

# Use grid() to complete the User Interface
title_label.grid(column=0, row=0, pady=40)
open_btn = Button(text="Select PDF", font=(WORD_FONT, 25), width=15, fg=WORD_COLOR, bg=BTN_BACKGROUND_COLOR,
                  command=text_speech)
open_btn.grid(column=0, row=1, pady=5)
root.mainloop()

2023-04-16 13:06:38.536 python[5900:128118] +[CATransaction synchronize] called within transaction


Good Things
I once heard an old man say,
Shaping vases out of clay
Into subtle forms sublime,
“Listen, son, good things take time.”
All my life I’ve thought of this
When a task was lacking bliss,
When the work seemed awfully tough
And I thought I’d had enough.
So I’d give a little more
To what sometimes seemed a chore;
And, you know, without a doubt,
Good things always came about.
~ Barbara Vance

Starting with the speech generation


Found cached dataset cmu-arctic-xvectors (/Users/andrea.huerfano/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f)


The audio was generated. See You!!
